In [58]:
import pandas as pd
url_users ="https://raw.githubusercontent.com/Akienobob/Masterschool/refs/heads/main/users.csv"
url_flights ="https://raw.githubusercontent.com/Akienobob/Masterschool/refs/heads/main/flights.csv"
url_sessions ="https://raw.githubusercontent.com/Akienobob/Masterschool/refs/heads/main/sessions.csv"
url_hotels ="https://raw.githubusercontent.com/Akienobob/Masterschool/refs/heads/main/hotels.csv"


users = pd.read_csv(url_users)
flights = pd.read_csv(url_flights)
sessions = pd.read_csv(url_sessions)
hotels = pd.read_csv(url_hotels)


# SESSIONS

In [59]:
# Die Session-Zeit in datetime konvertieren und die Sessiondauer berechnen
sessions['session_start'] = pd.to_datetime(sessions['session_start'])
sessions['session_end'] = pd.to_datetime(sessions['session_end'], format='mixed')
sessions['session_dauer']= sessions['session_end'] - sessions['session_start']

# Preise für Hotels und Flüge berechnen
hotels['price'] = hotels['rooms']*hotels['hotel_per_room_usd']*hotels['nights']
flights['price'] = flights['seats']*flights['base_fare_usd']

# negative Nächte wie -1 und -2 werden positiv. Insgesamt 105 von ca. 14K, daher keine große Auswirkung auf die Analyse
hotels['nights'] = hotels['nights'].apply(lambda x : abs(x) if x<0 else x)

In [60]:
hotels

,trip_id,hotel_name,nights,rooms,check_in_time,check_out_time,hotel_per_room_usd,price
0,582979-5475b9940ee240bf9328a0cc0bc9900c,NH Hotel - ottawa,6,1,2023-03-31 13:32:21.03,2023-04-07 11:00:00,110,660
1,514785-863ac3b4eb984f1eb226d67955623a5d,Choice Hotels - montreal,3,1,2023-06-21 20:06:24.3,2023-06-25 11:00:00,87,261
2,588386-233db0f627cf4aaaaeaa9a632daa5ec4,Accor - philadelphia,1,1,2023-04-04 10:43:44.175,2023-04-05 11:00:00,126,126
3,600395-79e934e3dcf2462587a0beffbebd6783,Marriott - memphis,1,2,2023-04-21 15:58:28.2,2023-04-23 11:00:00,153,306
4,474120-a87f749cf54b4f4ca9262d07080a06b2,Accor - san francisco,4,2,2023-02-02 14:18:11.205,2023-02-07 11:00:00,346,2768
...,...,...,...,...,...,...,...,...
14308,512377-acf089357a6b497d91eda34bb532578d,Marriott - washington,5,1,2023-04-02 12:25:51.465,2023-04-08 11:00:00,446,2230
14309,510580-8e74d152377f43aeb23bbed9ff17deaa,Crowne Plaza - montreal,1,1,2023-07-03 11:33:49.905,2023-07-04 11:00:00,69,69
14310,497042-fa163de72e234eaf9c9bc8b477fbf66a,Hyatt - san antonio,2,1,2023-04-13 11:40:58.35,2023-04-15 11:00:00,103,206
14311,598533-40848e5db09e46aa8f56c0c3ccdefb2d,Hyatt - macau,12,1,2024-01-24 09:54:31.635,2024-02-05 11:00:00,114,1368


# FEATURES GROUPBY

In [61]:
flights['departure_time'] = pd.to_datetime(flights['departure_time'])
flights['return_time'] = pd.to_datetime(flights['return_time'])

In [62]:
reisen_gesamt = sessions[['trip_id', 'user_id']].groupby('user_id').count()
reisen_gesamt.columns = ['reisen_gesamt']
reisen_gesamt

,reisen_gesamt
user_id,
23557,2
94883,2
101486,2
101961,5
106907,2
...,...
792549,4
796032,4
801660,3


In [63]:
stornierte_reisen = sessions[sessions['cancellation']][['trip_id','user_id']].groupby('user_id').count()
stornierte_reisen.columns = ['stornierte_reisen']

stornierte_reisen_id = sessions[sessions['cancellation']]['trip_id'].unique()
stornierte_reisen

,stornierte_reisen
user_id,
106907,1
171470,2
174997,1
182191,1
204943,1
...,...
736075,1
752933,1
763129,1


# Erfolgreiche Sessions- und Dauer ermitteln, die zu einer nicht stornierten Reise geführt haben

In [32]:

sessions_reisen = sessions[~sessions['trip_id'].isin(stornierte_reisen_id)].drop(columns=['cancellation'])

In [33]:
session_dauer_erfolg_avg = sessions_reisen.groupby('user_id')['session_dauer'].mean()

In [71]:

users['birthdate'] = pd.to_datetime(users['birthdate'])

today = pd.Timestamp.today()
# Alter der Reisenden anhand ihres Geburtstags und des heutigen Datums bestimmen
users['age']= (today-users['birthdate']).dt.days//365
users['user_since'] = ((today-pd.to_datetime(users['sign_up_date'])).dt.days//30.4).astype('int64')


In [72]:
user_trip_id = sessions_reisen[['user_id', 'trip_id']].drop_duplicates()

In [73]:
hotels = pd.merge(hotels,user_trip_id, on='trip_id', how='left')
hotels_valid = hotels[~hotels['trip_id'].isin(stornierte_reisen_id)].copy()
hotels_valid

,trip_id,hotel_name,nights,rooms,check_in_time,check_out_time,hotel_per_room_usd,price,user_id_x,user_id_y
0,582979-5475b9940ee240bf9328a0cc0bc9900c,NH Hotel - ottawa,6,1,2023-03-31 13:32:21.03,2023-04-07 11:00:00,110,660,582979.0,582979.0
1,514785-863ac3b4eb984f1eb226d67955623a5d,Choice Hotels - montreal,3,1,2023-06-21 20:06:24.3,2023-06-25 11:00:00,87,261,514785.0,514785.0
2,588386-233db0f627cf4aaaaeaa9a632daa5ec4,Accor - philadelphia,1,1,2023-04-04 10:43:44.175,2023-04-05 11:00:00,126,126,588386.0,588386.0
3,600395-79e934e3dcf2462587a0beffbebd6783,Marriott - memphis,1,2,2023-04-21 15:58:28.2,2023-04-23 11:00:00,153,306,600395.0,600395.0
4,474120-a87f749cf54b4f4ca9262d07080a06b2,Accor - san francisco,4,2,2023-02-02 14:18:11.205,2023-02-07 11:00:00,346,2768,474120.0,474120.0
...,...,...,...,...,...,...,...,...,...,...
14306,525053-971b75befda441c7b1af7500e4652ddf,Hilton - milwaukee,1,1,2023-03-31 20:42:13.815,2023-04-02 11:00:00,200,200,525053.0,525053.0
14307,583623-70097bf1d1334cfb891ac2bd09f448ae,Shangri-La - montreal,3,1,2023-02-23 17:25:07.185,2023-02-27 11:00:00,142,426,583623.0,583623.0
14309,510580-8e74d152377f43aeb23bbed9ff17deaa,Crowne Plaza - montreal,1,1,2023-07-03 11:33:49.905,2023-07-04 11:00:00,69,69,510580.0,510580.0
14310,497042-fa163de72e234eaf9c9bc8b477fbf66a,Hyatt - san antonio,2,1,2023-04-13 11:40:58.35,2023-04-15 11:00:00,103,206,497042.0,497042.0


# User Features überlegen - Durchschnittswerte- und Preise für Räume, Hotels, Flüge und Gepäck

In [65]:
user_rooms = hotels_valid.groupby('user_id')['rooms'].mean()
user_nights = hotels_valid.groupby('user_id')['nights'].mean()

user_price = hotels_valid.groupby('user_id')['price'].mean()
user_price = user_price.rename('hotel_price')

user_hotels = hotels_valid.groupby('user_id')['trip_id'].count()
user_hotels = user_hotels.rename('hotel_count')

In [66]:
flights = flights.merge(user_trip_id,on='trip_id')
flights_valid = flights[~flights['trip_id'].isin(stornierte_reisen_id)].copy()

In [69]:
user_seats = flights_valid.groupby('user_id')['seats'].mean()

user_flight_price = flights_valid.groupby('user_id')['base_fare_usd'].mean()
user_flight_price = user_flight_price.rename('flight_price')

user_flights = flights_valid.groupby('user_id')['trip_id'].count()
user_flights = user_flights.rename('flight_count')

user_checked_bags = flights_valid.groupby('user_id')['checked_bags'].mean()

# JOIN FEATURES

In [74]:
user_features = users[['user_id', 'gender', 'married', 'has_children', 'age','user_since']].copy()
user_features['user_id'].count()

5998

In [75]:
users

,user_id,birthdate,gender,married,has_children,home_country,home_city,home_airport,home_airport_lat,home_airport_lon,sign_up_date,age,user_since
0,531931,1984-05-26,F,False,False,usa,san jose,SJC,37.362,-121.929,2023-01-15,40,25
1,611065,1981-08-20,F,False,False,usa,los angeles,LAX,33.942,-118.408,2023-02-22,43,23
2,586762,1987-09-26,F,False,False,canada,montreal,YUL,45.468,-73.741,2023-02-11,37,24
3,229330,1976-11-08,F,True,False,usa,colorado springs,COS,38.806,-104.700,2022-06-27,48,31
4,536035,1995-11-09,M,False,False,usa,new york,LGA,40.777,-73.872,2023-01-17,29,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5993,591155,1982-12-26,M,True,False,canada,windsor,YQG,42.276,-82.956,2023-02-13,42,24
5994,521235,1989-06-21,F,False,False,usa,seattle,SEA,47.449,-122.309,2023-01-10,35,25
5995,600351,1976-07-15,F,True,False,usa,portland,PDX,45.589,-122.597,2023-02-17,48,23
5996,577044,1969-07-26,F,True,True,usa,chicago,UGN,42.422,-87.868,2023-02-06,55,24


# User Features in eine Tabelle bzw. einen Dataframe mergen

In [76]:
user_features = user_features.merge(reisen_gesamt, on='user_id', how='left').merge(stornierte_reisen,on='user_id', how='left').merge(session_dauer_erfolg_avg,on='user_id', how='left').merge(user_rooms,on='user_id', how='left').merge(user_nights,on='user_id', how='left').merge(user_price,on='user_id', how='left').merge(user_hotels,on='user_id', how='left').merge(user_seats,on='user_id', how='left').merge(user_flight_price,on='user_id', how='left').merge(user_flights,on='user_id', how='left').merge(user_checked_bags,on='user_id', how='left')

In [77]:
user_features.describe()

,user_id,age,user_since,reisen_gesamt,stornierte_reisen,session_dauer,rooms,nights,hotel_price,hotel_count,seats,flight_price,flight_count,checked_bags
count,5998.000000,5998.000000,5998.000000,5998.000000,595.000000,5998,5336.000000,5336.000000,5336.000000,5336.000000,5045.000000,5045.000000,5045.000000,5045.000000
mean,545440.695565,42.453151,24.437979,2.784595,1.025210,0 days 00:01:55.249517365,1.199139,3.735627,810.108972,2.604198,1.185950,462.988802,2.607929,0.578599
std,64924.749760,12.055388,1.207394,1.570621,0.156895,0 days 00:00:35.667593038,0.353380,2.852379,949.442129,1.279649,0.370965,441.530541,1.287289,0.460129
min,23557.000000,18.000000,21.000000,0.000000,1.000000,0 days 00:00:30.666666666,1.000000,0.000000,-477.000000,1.000000,1.000000,5.350000,1.000000,0.000000
25%,517152.000000,35.000000,24.000000,2.000000,1.000000,0 days 00:01:30.555555555,1.000000,2.000000,304.900000,2.000000,1.000000,276.243333,2.000000,0.250000
50%,540542.000000,43.000000,24.000000,3.000000,1.000000,0 days 00:01:51.500000,1.000000,3.000000,544.000000,2.000000,1.000000,385.866667,2.000000,0.500000
75%,574146.000000,50.000000,25.000000,4.000000,1.000000,0 days 00:02:16,1.333333,4.750000,957.916667,3.000000,1.250000,536.445000,3.000000,1.000000
max,844489.000000,89.000000,42.000000,9.000000,2.000000,0 days 00:07:11.333333333,4.000000,30.000000,14057.000000,8.000000,6.000000,9135.130000,8.000000,5.000000


In [90]:
user_features['stornierte_reisen'] = user_features['stornierte_reisen'].fillna(0)
user_features['flight_count'] = user_features['flight_count'].fillna(0)
user_features['hotel_count'] = user_features['hotel_count'].fillna(0)

In [91]:
user_features.head(10)

,user_id,gender,married,has_children,age,user_since,reisen_gesamt,stornierte_reisen,rooms_avg,nights_avg,hotel_price_avg,hotel_count,seats_avg,flight_price_avg,flight_count,checked_bags_avg,session_dauer_avg_sec
0,531931,1,False,False,40,25,2,0.0,1.500000,2.500000,492.500000,2.0,1.500000,944.145000,2.0,0.500000,70.875
1,611065,1,False,False,43,23,4,0.0,1.500000,1.250000,407.500000,4.0,1.500000,893.192500,4.0,0.750000,146.750
2,586762,1,False,False,37,24,3,0.0,1.333333,6.333333,1939.666667,3.0,1.333333,607.430000,3.0,0.333333,116.375
3,229330,1,True,False,48,31,4,0.0,1.750000,1.500000,798.250000,4.0,1.750000,387.027500,4.0,0.250000,119.625
4,536035,0,False,False,29,25,2,0.0,1.500000,1.000000,163.000000,2.0,1.500000,488.340000,2.0,0.500000,102.750
5,477730,1,False,False,46,25,5,0.0,1.200000,3.600000,1074.000000,5.0,1.200000,697.142000,5.0,0.000000,171.750
6,513030,1,False,False,39,25,3,0.0,1.000000,4.000000,720.333333,3.0,1.000000,202.276667,3.0,0.000000,91.500
7,570821,1,True,True,42,24,3,0.0,1.000000,2.666667,294.666667,3.0,1.000000,482.623333,3.0,0.666667,88.250
8,517076,1,False,False,39,25,5,0.0,1.200000,6.800000,890.000000,5.0,1.000000,459.085000,2.0,0.000000,196.750
9,585248,1,True,True,49,24,1,0.0,1.000000,1.000000,121.000000,1.0,1.000000,394.550000,1.0,0.000000,81.875


In [79]:
user_features.rename(columns={'session_dauer':'session_dauer_avg', 'rooms': 'rooms_avg', 'nights':'nights_avg', 'hotel_price':'hotel_price_avg', 'seats':'seats_avg', 'flight_price':'flight_price_avg', 'checked_bags':'checked_bags_avg'}, inplace= True)

In [81]:
user_features['session_dauer_avg'].dt.total_seconds()
user_features['session_dauer_avg_sec'] = user_features['session_dauer_avg'].dt.total_seconds()

In [82]:
user_features['gender'] = user_features['gender'].replace({'F':1, 'M':0, 'O':2})

In [84]:
user_features = user_features.drop(columns='session_dauer_avg')

Die entstandene User-Features Tabelle als .csv Datei exportieren - weitere Bearbeitung und Clustering im folgenden Notebook: [TravelTideEDAundClustering](https://github.com/Akienobob/Masterschool/blob/main/TravelTideEDAundClustering.ipynb)

In [86]:
#user_features.to_csv('user_features.csv', index_label=False)

In [92]:
user_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5998 entries, 0 to 5997
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   user_id                5998 non-null   int64  
 1   gender                 5998 non-null   int64  
 2   married                5998 non-null   bool   
 3   has_children           5998 non-null   bool   
 4   age                    5998 non-null   int64  
 5   user_since             5998 non-null   int64  
 6   reisen_gesamt          5998 non-null   int64  
 7   stornierte_reisen      5998 non-null   float64
 8   rooms_avg              5336 non-null   float64
 9   nights_avg             5336 non-null   float64
 10  hotel_price_avg        5336 non-null   float64
 11  hotel_count            5998 non-null   float64
 12  seats_avg              5045 non-null   float64
 13  flight_price_avg       5045 non-null   float64
 14  flight_count           5998 non-null   float64
 15  chec

In [93]:
user_features.mean()

,0
user_id,545440.695565
gender,0.885962
married,0.440480
has_children,0.326442
age,42.453151
user_since,24.437979
reisen_gesamt,2.784595
stornierte_reisen,0.101701
rooms_avg,1.199139
nights_avg,3.735627


In [89]:
user_features['flight_count'].max()

8.0

Analyse Discounts - noch nicht im Projekt enthalten. Zu klärende Fragen: wie kommt es zu Discounts (Aktionen, Werbung, Vorteile für bestimmte User-Gruppen?)

In [22]:
#discounts = session_merge[["user_id","flight_discount", "flight_discount_amount", "hotel_discount","hotel_discount_amount","nights","rooms", "flight_booked","cancellation"]]
#discounts

,user_id,flight_discount,flight_discount_amount,hotel_discount,hotel_discount_amount,nights,rooms,flight_booked,cancellation
0,483982,False,NaN,False,NaN,NaN,NaN,False,False
1,491175,False,NaN,False,NaN,NaN,NaN,False,False
2,537580,False,NaN,False,NaN,NaN,NaN,False,False
3,541084,True,0.2,False,NaN,7.0,1.0,False,False
4,588820,False,NaN,False,NaN,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...
49206,542095,False,NaN,False,NaN,7.0,1.0,True,False
49207,553943,False,NaN,False,NaN,NaN,NaN,True,False
49208,569792,False,NaN,False,NaN,6.0,1.0,True,False
49209,552809,False,NaN,False,NaN,NaN,NaN,True,False


In [24]:
#discounts

,user_id,flight_discount,flight_discount_amount,hotel_discount,hotel_discount_amount,nights,rooms,flight_booked,cancellation
0,483982,0,NaN,0,NaN,NaN,NaN,0,0
1,491175,0,NaN,0,NaN,NaN,NaN,0,0
2,537580,0,NaN,0,NaN,NaN,NaN,0,0
3,541084,1,0.2,0,NaN,7.0,1.0,0,0
4,588820,0,NaN,0,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...
49206,542095,0,NaN,0,NaN,7.0,1.0,1,0
49207,553943,0,NaN,0,NaN,NaN,NaN,1,0
49208,569792,0,NaN,0,NaN,6.0,1.0,1,0
49209,552809,0,NaN,0,NaN,NaN,NaN,1,0


In [25]:
#discounts[discounts["flight_booked"]==1]

,user_id,flight_discount,flight_discount_amount,hotel_discount,hotel_discount_amount,nights,rooms,flight_booked,cancellation
70,512201,0,NaN,1,0.1,8.0,1.0,1,0
101,517491,0,NaN,0,NaN,3.0,1.0,1,0
332,488865,0,NaN,0,NaN,2.0,1.0,1,0
415,560978,0,NaN,0,NaN,16.0,1.0,1,0
783,542012,1,0.05,0,NaN,1.0,1.0,1,0
...,...,...,...,...,...,...,...,...,...
49205,553139,1,0.30,0,NaN,5.0,2.0,1,0
49206,542095,0,NaN,0,NaN,7.0,1.0,1,0
49207,553943,0,NaN,0,NaN,NaN,NaN,1,0
49208,569792,0,NaN,0,NaN,6.0,1.0,1,0
